In [15]:
# TODO: Determine position size: Dollar Volatility = N × Dollars per Point. Unit = 1% of porfolio / Dollar Volatility
# TODO: Reduce trading size when account amount decreases (10% overall decrease -> 20% trading amount decrease), reinstate when back to initial starting amount
# TODO: Allow for both short and longs
# TODO: Implement System 2
# TODO: Ignoring breakouts (System 1) as previous one was a winner (regardless of long or short), failsafe is System 2 entry (System 2 does not have this breakout filter)
# Implemented: System 1 Longs, pyramiding up to 5 additional orders (1/2 N increments)
# Implementd: fixed selling over multiple orders (observation: some work better when selling over a period of time vs all at once)

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.long = None
        self.pyramid = 0
        self.previousOrder = None
        self.totalNumberOfOrders = 0
        self.selling = False
        
        # 20 day ATR for N value
        self.atr = bt.indicators.ATR(self.datas[0], period = 20)
        
        # System 1: 20 day entry, 10 day exit
        self.entry = bt.indicators.Highest(period=20)
        self.exit = bt.indicators.Lowest(period=10)
        
        # System 2: 55 day entry, 20 day exit

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.previousOrder = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.entry[-1] < self.dataopen[0]:

                    # BUY, BUY, BUY!!! (with default parameters)
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.log('entry high: %.2f' % self.entry[-1])

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy()
                    self.selling = False
                    #self.long = True

        else:
            # in the market, see if we can pyramid our position
            if self.pyramid < 5 and self.dataopen[0] > (self.previousOrder + 1/2*self.atr[-1]) and not self.selling:
                # BUY, BUY, BUY!!! (with default parameters)
                self.log('BUY CREATE PYRAMID, current close price: %.2f' % self.dataclose[0])
                self.log('N value: %.2f' % self.atr[-1])
                self.log('Pyramid: %.2f' % self.pyramid)
                self.log('N buy in price: %.2f' % (self.previousOrder + 1/2*self.atr[-1]))
                
                self.pyramid+=1
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
                self.log('New previousOrder, should be same as BUY EXECUTED: %.2f' % self.previousOrder)
                

            # Already in the market ... we might sell
            if self.exit[-1] > self.dataopen[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                # Below system low exit price
                self.log('SELL CREATE, 10 day low: %.2f' % self.exit[-1])
                self.log('exit low: %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.pyramid = 0
                for data in self.datas:
                    size=self.getposition(data).size
                    if  size!=0:
                        self.order = self.close(data)
                # self.order = self.sell()
                self.selling = True
                
            elif self.dataopen[0] < (self.dataclose[-1]-2*self.atr[-1]):
                # Below 2N stop
                stopPrice = self.dataclose[-1]-2*self.atr[-1]
                self.log('SELL CREATE, N stop: %.2f' % stopPrice)
                self.log('Current Price: %.2f' % self.dataclose[0])
                self.log('N value: %.2f' % self.atr[-1])
                # Keep track of the created order to avoid a 2nd order
                self.pyramid = 0
                for data in self.datas:
                    size=self.getposition(data).size
                    if  size!=0:
                        self.order = self.close(data)
                # self.order = self.sell()
                self.selling = True


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname='/Users/Ziyi/Documents/backtrader/data/BTC-USD.csv', 
        #  fromdate=datetime.datetime(2019, 1, 1), 
        #  todate=datetime.datetime(2019, 12, 31),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

1133.66
2017-02-24, New previousOrder, should be same as BUY EXECUTED: 1117.27
2017-02-25, BUY EXECUTED, Price: 1170.41, Cost: 11704.10, Comm 11.70
2017-03-02, BUY CREATE PYRAMID, current close price: 1251.01
2017-03-02, N value: 34.38
2017-03-02, Pyramid: 4.00
2017-03-02, N buy in price: 1187.60
2017-03-02, New previousOrder, should be same as BUY EXECUTED: 1170.41
2017-03-03, BUY EXECUTED, Price: 1250.71, Cost: 12507.10, Comm 12.51
2017-03-11, SELL CREATE, 10 day low: 1116.72
2017-03-11, exit low: 1175.83
2017-03-12, SELL EXECUTED, Price: 1176.62, Cost: 66244.20, Comm 70.60
2017-03-12, OPERATION PROFIT, GROSS 4353.00, NET 4216.16
2017-04-19, BUY CREATE, 1210.29
2017-04-19, entry high: 1211.67
2017-04-20, BUY EXECUTED, Price: 1211.08, Cost: 12110.80, Comm 12.11
2017-04-23, BUY CREATE PYRAMID, current close price: 1207.21
2017-04-23, N value: 35.85
2017-04-23, Pyramid: 0.00
2017-04-23, N buy in price: 1229.01
2017-04-23, New previousOrder, should be same as BUY EXECUTED: 1211.08
2017-0